In [1]:
import sys
sys.path.append("C:/Users/javie/Desktop/CVC")

import os
from torch.utils.data import DataLoader, random_split, Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
import pickle
import torch
import trimesh
import numpy as np
from torch import nn
from tqdm.notebook import tnrange, tqdm
import time

from utils.slices import *
from utils.orientaciones import *


In [2]:
class CustomImageDataset(Dataset):
    def __init__(self, data_dir, pkl_dir, transform=None):
        self.data_dir = data_dir
        self.transform = transform
        self.pkl_dir = pkl_dir

        self.classes = os.listdir(data_dir)
        self.mesh_info = []
        self.labels = []
        self.n = 0
        for class_index, class_path in enumerate(self.classes):
            # Create directory if does not exist
            os.makedirs(pkl_dir + class_path, exist_ok=True)
            mesh_paths = os.listdir(data_dir + class_path)
            for path in mesh_paths:
                pkl_path = pkl_dir + class_path + '/' + path + '.pkl'
                if os.path.exists(pkl_path):
                    with open(pkl_path, 'rb') as archivo:
                        pkl_object = pickle.load(archivo)
                    #self.classes.append(pkl_object[0])
                    self.mesh_info.append(pkl_object[1])
                    self.labels.append(pkl_object[2]) #index of self.classes
                else:
                    mesh = trimesh.load(data_dir + class_path + '/' + path)
                    descriptors_list = sample_slices(mesh)
                    
                    self.mesh_info.append(descriptors_list)
                    self.labels.append(class_index)
                    with open(pkl_path, 'wb') as archivo:
                        pickle.dump([class_path, descriptors_list, class_index], archivo)

        #self.classes = torch.tensor(self.classes)
        self.mesh_info = torch.tensor(self.mesh_info)
        self.labels = torch.tensor(self.labels)
            
        
    def __len__(self):
        return len(self.labels)

    def __getitem__(self, index):
        return self.mesh_info[index], self.labels[index]

In [3]:
dataset_path = "../data/oriented_dataset/"
pkl_path = "../data/descriptors_dataset/"
dataset = CustomImageDataset(dataset_path, pkl_path)

In [4]:
# 2ROW / 6ROW adjustment
dataset.labels[dataset.labels == 0] = 0  # Dundee 2ROW British
dataset.labels[dataset.labels == 1] = 0  # Dundee 2ROW Scottish
dataset.labels[dataset.labels == 2] = 1  # Dundee 6ROW BERE Orkney
dataset.labels[dataset.labels == 3] = 1  # Dundee 6ROW BERE Unknown
dataset.labels[dataset.labels == 4] = 1  # Dundee 6ROW BERE Western Isles
dataset.labels[dataset.labels == 5] = 1  # Dundee 6ROW Faro
dataset.labels[dataset.labels == 6] = 1  # Dundee 6ROW Scandinavian
dataset.labels[dataset.labels == 7] = 0  # Orkney 2ROW British
dataset.labels[dataset.labels == 8] = 0  # Orkney 2ROW Scottish
dataset.labels[dataset.labels == 9] = 1  # Orkney 6ROW BERE Orkney
dataset.labels[dataset.labels == 10] = 1  # Orkney 6ROW BERE Unknown
dataset.labels[dataset.labels == 11] = 1  # Orkney 6ROW BERE Western Isles
dataset.labels[dataset.labels == 12] = 1  # Orkney 6ROW Scandinavian


In [5]:
train_size = int(0.7 * len(dataset))
test_size = len(dataset) - train_size

train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

batch_size = 16
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [6]:
for X, y in train_loader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")

Shape of X [N, C, H, W]: torch.Size([16, 904])
Shape of y: torch.Size([16]) torch.int64
Shape of X [N, C, H, W]: torch.Size([16, 904])
Shape of y: torch.Size([16]) torch.int64
Shape of X [N, C, H, W]: torch.Size([16, 904])
Shape of y: torch.Size([16]) torch.int64
Shape of X [N, C, H, W]: torch.Size([16, 904])
Shape of y: torch.Size([16]) torch.int64
Shape of X [N, C, H, W]: torch.Size([16, 904])
Shape of y: torch.Size([16]) torch.int64
Shape of X [N, C, H, W]: torch.Size([16, 904])
Shape of y: torch.Size([16]) torch.int64
Shape of X [N, C, H, W]: torch.Size([16, 904])
Shape of y: torch.Size([16]) torch.int64
Shape of X [N, C, H, W]: torch.Size([16, 904])
Shape of y: torch.Size([16]) torch.int64
Shape of X [N, C, H, W]: torch.Size([16, 904])
Shape of y: torch.Size([16]) torch.int64
Shape of X [N, C, H, W]: torch.Size([16, 904])
Shape of y: torch.Size([16]) torch.int64
Shape of X [N, C, H, W]: torch.Size([16, 904])
Shape of y: torch.Size([16]) torch.int64
Shape of X [N, C, H, W]: torch.S

In [7]:
'''# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")
'''
# Define model
class GrainClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(904, 512, dtype=torch.double),
            nn.ReLU(),
            nn.Linear(512, 512, dtype=torch.double),
            nn.ReLU(),
            nn.Linear(512, 512, dtype=torch.double),
            nn.ReLU(),
            nn.Linear(512, 512, dtype=torch.double),
            nn.ReLU(),
            nn.Linear(512, 512, dtype=torch.double),
            nn.ReLU(),
            nn.Linear(512, 512, dtype=torch.double),
            nn.ReLU(),
            nn.Linear(512, 512, dtype=torch.double),
            nn.ReLU()
        )
        self.classification_layer = nn.Linear(512, 2, dtype=torch.double)
    def forward(self, x):
        x = self.flatten(x)
        x = self.linear_relu_stack(x)
        x = self.classification_layer(x)
        return x

'''model = GrainClassifier().to(device)
print(model)'''

model = GrainClassifier()
print(model)

GrainClassifier(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=904, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=512, bias=True)
    (5): ReLU()
    (6): Linear(in_features=512, out_features=512, bias=True)
    (7): ReLU()
    (8): Linear(in_features=512, out_features=512, bias=True)
    (9): ReLU()
    (10): Linear(in_features=512, out_features=512, bias=True)
    (11): ReLU()
    (12): Linear(in_features=512, out_features=512, bias=True)
    (13): ReLU()
  )
  (classification_layer): Linear(in_features=512, out_features=2, bias=True)
)


In [8]:
'''
    X = torch.rand(10, 124, device=device)
    X *= -1
    logits = model(X)
    pred_probab = nn.Softmax(dim=1)(logits)
    y_pred = pred_probab.argmax(1)
    print(f"Input: {X}")
    print(f"Prob class: {pred_probab}")
    print(f"Predicted class: {y_pred}")'''

'\n    X = torch.rand(10, 124, device=device)\n    X *= -1\n    logits = model(X)\n    pred_probab = nn.Softmax(dim=1)(logits)\n    y_pred = pred_probab.argmax(1)\n    print(f"Input: {X}")\n    print(f"Prob class: {pred_probab}")\n    print(f"Predicted class: {y_pred}")'

In [9]:
# Optimizers specified in the torch.optim package
loss_fn = torch.nn.CrossEntropyLoss()
learning_rate = 1e-3
momentum = 0.9
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=momentum)

In [10]:
def train_one_epoch(dataloader):
    running_loss = 0.
    # Here, we use enumerate(training_loader) instead of
    # iter(training_loader) so that we can track the batch
    # index and do some intra-epoch reporting
    for i, data in enumerate(dataloader):
        # Every data instance is an input + label pair
        inputs, labels = data

        # Zero your gradients for every batch!
        optimizer.zero_grad()

        # Make predictions for this batch
        outputs = model(inputs)
        '''print("outputs")
        print(outputs)
        print("labels")
        print(labels)
        '''# Compute the loss and its gradients
        loss = loss_fn(outputs, labels)
        loss.backward()

        # Adjust learning weights
        optimizer.step()

        # Gather data and report
        running_loss += loss.item()
    #print("Total loss: ", running_loss)

    return running_loss

In [11]:
def test_model(dataloader):
    total_loss = 0.0
    correct = 0
    total = 0
    for i, data in enumerate(dataloader):
        inputs, labels = data
        outputs = model(inputs)

        loss = loss_fn(outputs, labels)
        total_loss += loss.item()

        _, predictions = torch.max(outputs.data, 1)
        correct += (predictions == labels).sum().item()
        total += labels.size(0)  # Número total de ejemplos

    # Calcula la precisión como el número de predicciones correctas dividido por el total
    accuracy = correct / total
        
    return total_loss, accuracy

In [12]:
save_path = "../models/old_school"
max_accuracy = 0
for i in range(1000):
    train_loss = train_one_epoch(train_loader)
    test_loss, accuracy = test_model(test_loader)
    if max_accuracy < accuracy:
        max_accuracy = accuracy
        print("MAX ACCURACY")
        print("Epoch: " + "%-3i" % i + ", train loss: " + str(train_loss) + ", test loss: " + str(test_loss), ", accuracy: " + str(accuracy))
        torch.save(model.state_dict(), save_path)
        continue
    if i % 10 == 0:
        print("Epoch: " + "%-3i" % i + ", train loss: " + str(train_loss) + ", test loss: " + str(test_loss), ", accuracy: " + str(accuracy))
print(max_accuracy)

MAX ACCURACY
Epoch: 0  , train loss: 21.501395584761813, test loss: 9.710076731458464 , accuracy: 0.5238095238095238
MAX ACCURACY
Epoch: 10 , train loss: 21.47677484612678, test loss: 9.69555431878521 , accuracy: 0.5666666666666667
MAX ACCURACY
Epoch: 11 , train loss: 21.473978127812227, test loss: 9.695334261975205 , accuracy: 0.6476190476190476
Epoch: 20 , train loss: 21.461593773390437, test loss: 9.687719520686619 , accuracy: 0.49047619047619045
Epoch: 30 , train loss: 21.44743816505955, test loss: 9.678232645563773 , accuracy: 0.49047619047619045
MAX ACCURACY
Epoch: 37 , train loss: 21.422110421959257, test loss: 9.670639692082972 , accuracy: 0.6666666666666666
Epoch: 40 , train loss: 21.43238292867368, test loss: 9.665525432244698 , accuracy: 0.5904761904761905
MAX ACCURACY
Epoch: 49 , train loss: 21.373167872315417, test loss: 9.645904294937885 , accuracy: 0.6714285714285714
Epoch: 50 , train loss: 21.371692183806626, test loss: 9.642976308666904 , accuracy: 0.6476190476190476
M

In [15]:
mesh = trimesh.load("../data/oriented_dataset/Orkney 2ROW British/OHBT16104 7L10.stl")
descriptors_list = sample_slices(mesh)

In [28]:
inp = torch.tensor(descriptors_list).unsqueeze(0)
inp.size()

torch.Size([1, 904])

In [29]:
outputs = model(inp)
print(f"outputs = {outputs}")
_, predictions = torch.max(outputs.data, 1)
print(f"predictions = {predictions}")

outputs = tensor([[ 1.7967, -1.6297]], dtype=torch.float64, grad_fn=<AddmmBackward0>)
predictions = tensor([0])
